### Fixed Traction BCs

In [7]:
from ipynb.fs.full.Background_Functions import *
from ipynb.fs.full._04_Fixed_Displacement_BCs import *

#### Final notebook output: 






In [2]:
class boundaryCellTraction(A):

    def __init__(self, edges, xy):
        
        sw_S, sw_N, sw_W, sw_E = 1, 1, 1, 1

        for edge in edges:
            if edge == "b": sw_S = 0
            if edge == "t": sw_N = 0
            if edge == "l": sw_W = 0
            if edge == "r": sw_E = 0
        
        if transient:
            self.a_P = (rho*dx*dy/(dt**2)) + A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W
        else:
            self.a_P = A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W

        self.a_N = A(xy).a_N 
        self.a_S = A(xy).a_S 
        self.a_E = A(xy).a_E 
        self.a_W = A(xy).a_W 

        # self.a_faces = A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W 

    def b_temp(u_old, u_old_old, k, xy):      
        return A.b_temp(u_old, u_old_old, k, xy)
    
    def b_diff(edges, k, xy, u_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                    (boundaryCellTraction.corner(edges, "NE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "NW", uv, u_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellTraction.corner(edges, "SE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SW", uv, u_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellTraction.corner(edges, "NE", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SE", uv, u_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellTraction.corner(edges, "NW", uv, u_previous, k) - boundaryCellTraction.corner(edges, "SW", uv, u_previous, k))
                        /dy)
                )

        for edge in edges:

            if (edge == "b") & (xy == "x") : S_term =  Sfy*tr_bottom_x  
            if (edge == "b") & (xy == "y") : S_term =  Sfy*tr_bottom_y  
            if (edge == "t") & (xy == "x") : N_term =  Sfy*tr_top_x  
            if (edge == "t") & (xy == "y") : N_term =  Sfy*tr_top_y  
            if (edge == "l") & (xy == "x") : W_term =  Sfx*tr_left_x  
            if (edge == "l") & (xy == "y") : W_term =  Sfx*tr_left_y  
            if (edge == "r") & (xy == "x") : E_term =  Sfx*tr_right_x  
            if (edge == "r") & (xy == "y") : E_term =  Sfx*tr_right_y        

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion
    
    def corner(edges, corner_placement, uv, u_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, u_previous, uv_i)

        if corner_placement == "NE":
            corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

        for edge in edges:
            if (edge == "b") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.S)
            if (edge == "b") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.S)

            if (edge == "t") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.N)
            if (edge == "t") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.N)

            if (edge == "l") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.W)
            if (edge == "l") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.W)

            if (edge == "r") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.E)
            if (edge == "r") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.E)
        
        return corner

# boundaryCellTraction.b_diff(["b", "l"], 14, "x", u_previous)
   

In [3]:
class boundaryPointTraction(A):

    def __init__(self, edges, xy):    # The order matters for edges, bottom left corner on the bottom is : ["b", "l"]
        
        sw_S, sw_N, sw_W, sw_E = 0, 0, 0, 0

        if edges[0] == "b": sw_S = 1
        if edges[0] == "t": sw_N = 1
        if edges[0] == "l": sw_W = 1
        if edges[0] == "r": sw_E = 1
        
        a_P = (A(xy).a_N*sw_N + A(xy).a_S**sw_S )*2/(Sfy) + (A(xy).a_E*sw_E + A(xy).a_W*sw_W)*2/(Sfx)
        
        a_N = A(xy).a_N*2/(Sfy) # are terms cancel in derivation
        a_S = A(xy).a_S*2/(Sfy)
        a_E = A(xy).a_E*2/(Sfx)
        a_W = A(xy).a_W*2/(Sfx)

        if edges[0] == "b":
            self.a_N = a_P
            self.a_P = a_S
        if edges[0] == "t":
            self.a_S = a_P
            self.a_P = a_N
        if edges[0] == "l":
            self.a_E = a_P
            self.a_P = a_W
        if edges[0] == "r":
            self.a_W = a_P
            self.a_P = a_E
    
    def b_diff(edges, k, xy, u_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        def N_term():
            
            N_term =( A.coef(xy, "N", uv)*(
                    (boundaryPointTraction.corner(edges, "E", uv, u_previous, k) - boundaryPointTraction.corner(edges, "W", uv, u_previous, k))
                    /dx)
            )

            return N_term

        def S_term():
            
            S_term =(
                    A.coef(xy, "S", uv)*(
                        (boundaryPointTraction.corner(edges, "E", uv, u_previous, k) - boundaryPointTraction.corner(edges, "W", uv, u_previous, k))
                        /dx)
                ) 
            return S_term

        def E_term():
            
            E_term =(
                    A.coef(xy, "E", uv)*(
                        (boundaryPointTraction.corner(edges, "N", uv, u_previous, k) - boundaryPointTraction.corner(edges, "S", uv, u_previous, k))
                        /dy)
                ) 
            return E_term
            
        def W_term():
            
            W_term =(
                        A.coef(xy, "W", uv)*(
                        (boundaryPointTraction.corner(edges, "N", uv, u_previous, k) - boundaryPointTraction.corner(edges, "S", uv, u_previous, k))
                        /dy)
                )
                
            return W_term

        # This can be tidied up/shortened

        if (edges[0] == "b") & (xy == "x") : b_diffusion =  tr_bottom_x  + N_term()
        if (edges[0] == "b") & (xy == "y") : b_diffusion =  tr_bottom_y  + N_term()
        if (edges[0] == "t") & (xy == "x") : b_diffusion =  tr_top_x + S_term()
        if (edges[0] == "t") & (xy == "y") : b_diffusion =  tr_top_y + S_term()
        if (edges[0] == "l") & (xy == "x") : b_diffusion =  tr_left_x + E_term()
        if (edges[0] == "l") & (xy == "y") : b_diffusion =  tr_left_y + E_term()
        if (edges[0] == "r") & (xy == "x") : b_diffusion =  tr_right_x + W_term()
        if (edges[0] == "r") & (xy == "y") : b_diffusion =  tr_right_y + W_term()  

        return b_diffusion
    
    def corner(edges, corner_placement, uv, u_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, u_previous, uv_i)


        if corner_placement == "E":
            corner =  (1/2)*(disp.E + disp.P)
        if corner_placement == "W":
            corner =  (1/2)*(disp.W + disp.P)
        if  corner_placement == "N":
            corner =  (1/2)*(disp.N + disp.P)
        if  corner_placement == "S":
            corner =  (1/2)*(disp.S + disp.P)

        if len(edges) > 1:

            if (edges[1] == "r") & (corner_placement == "E"):
                    corner =  disp.E
            if (edges[1] == "l") & (corner_placement == "W"):
                    corner =  disp.W
            if (edges[1] == "t") & (corner_placement == "N"):
                    corner =  disp.N
            if (edges[1] == "b") & (corner_placement == "S"):
                    corner =  disp.S
        
        return corner
# boundaryCellTraction.b_diff(["b", "l"], 14, "x", u_previous)
   

In [4]:
def traction_cell_BCs(A_matrix, b_matrix, k, edges, xy, u_old, u_old_old, u_previous):

    A_matrix[k,k] = boundaryCellTraction(edges, xy).a_P

    b_matrix[k] =(
                    boundaryCellTraction.b_temp(u_old, u_old_old, k, xy)
                    +
                    boundaryCellTraction.b_diff(edges, k, xy, u_previous)  
                )
    for edge in edges:
        if edge == "b":
            #an
            A_matrix[k,index(k).n] = - boundaryCellTraction(edges, xy).a_N

        if edge == "t":
            #as
            A_matrix[k,index(k).s] = - boundaryCellTraction(edges, xy).a_S

        if edge == "l":
            #ae
            A_matrix[k,index(k).e] = - boundaryCellTraction(edges, xy).a_E

        if edge == "r":
            #aw
            A_matrix[k,index(k).w] = - boundaryCellTraction(edges, xy).a_W           

    if len(edges) == 1:

        if (edges[0] == "b") | (edges[0] == "t") :
            #ae
            A_matrix[k, index(k).e] = - boundaryCellTraction(edges, xy).a_E
            #aw
            A_matrix[k, index(k).w] = - boundaryCellTraction(edges, xy).a_W

        if (edges[0] == "l") | (edges[0] == "r") :
            #an
            A_matrix[k,index(k).n] = - boundaryCellTraction(edges, xy).a_N            
            #as
            A_matrix[k,index(k).s] = - boundaryCellTraction(edges, xy).a_S

    return A_matrix, b_matrix

In [5]:
def traction_point_BCs(A_matrix, b_matrix, k, edges, xy, u_previous):
    
    #ap
    A_matrix[k,k] = boundaryPointTraction(edges, xy).a_P
    #b
    b_matrix[k] =boundaryPointTraction.b_diff(edges, k, xy, u_previous)

    if edges[0] == "b":
        #an
        A_matrix[k,index(k).n] = - boundaryPointTraction(edges, xy).a_N
    if edges[0] == "t":
        #as
        A_matrix[k,index(k).s] = - boundaryPointTraction(edges, xy).a_S
    if edges[0] == "l":
        #ae
        A_matrix[k, index(k).e] = - boundaryPointTraction(edges, xy).a_E
    if edges[0] == "r":
        #aw
        A_matrix[k, index(k).w] = -  boundaryPointTraction(edges, xy).a_W

    return A_matrix, b_matrix

In [6]:
rho

8050